In [2]:
import os
import openai
import sys
sys.path.append('../..')

import constants
openai.api_key = constants.OPENAI_API_KEY

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [4]:
embedding = OpenAIEmbeddings(openai_api_key=constants.OPENAI_API_KEY)
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [5]:
print(vectordb._collection.count())

104


In [6]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [7]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The document the is from, should be one of ./DB004.pdf",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the database",
        type="integer",
    ),
]

In [8]:
document_content_description = "Database notes"
llm = OpenAI(temperature=0, openai_api_key=constants.OPENAI_API_KEY)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [9]:
question = "What is TIS?"

In [10]:
docs = retriever.get_relevant_documents(question)

/Users/christianbraun/opt/miniconda3/envs/langchain-with-your-data/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='TIS' filter=None limit=None


In [11]:
for d in docs:
    print(d.metadata)

{'source': './DB004.pdf', 'page': 16}
{'source': './DB004.pdf', 'page': 16}
{'source': './DB004.pdf', 'page': 37}
{'source': './DB004.pdf', 'page': 37}


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor